In [1]:
def opp_goal_probs_from_cs(cs_prob):
    """
    Compute opponent goal probabilities given your team's clean sheet chance.
    """
    lam = -np.log(cs_prob)
    p0 = np.exp(-lam)
    p1 = lam * p0
    p2 = (lam**2) * p0 / 2
    p3 = (lam**3) * p0 / 6
    p23 = p2 + p3
    p4plus = 1 - (p0 + p1 + p2 + p3)

    return p23, p4plus

In [3]:
import requests
import pandas as pd
import numpy as np
# --- Get the data ---
r = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
data = r.json()

players = pd.DataFrame(data["elements"])
teams = pd.DataFrame(data["teams"])
positions = pd.DataFrame(data["element_types"])
minutes_df = pd.read_csv("/content/drive/MyDrive/fplreview_10.csv").rename(columns={"ID":"player_id","10_xMins":"xMin"})
cs_df = pd.read_csv("/content/drive/MyDrive/cs_10.csv")
cs_df["CS%"] = cs_df["CS%"].str.rstrip('%').astype(float) / 100
cs_df = cs_df.rename(columns={"Team": "team_name", "CS%": "xClean_sheets"})
cs_df["x2"], cs_df["x4"] = opp_goal_probs_from_cs(cs_df["xClean_sheets"])
# --- Save the player id before merging ---
players['player_id'] = players['id']
# --- Merge player info with position names and team names ---
players_full = (
    players
    .merge(positions[["id", "singular_name"]], left_on="element_type", right_on="id", how="left")
    .merge(teams[["id", "name"]], left_on="team", right_on="id", suffixes=("", "_team"))
)
players_full = players_full.merge(minutes_df[["player_id", "xMin"]], on="player_id", how="left", validate="1:1")
# Clean up the column names
players_full = players_full.rename(columns={
    "singular_name": "position",
    "name": "team_name"
})
players_full = players_full.merge(cs_df[["team_name", "xClean_sheets", "x2", "x4"]], on="team_name", how="left", validate="many_to_one")
# Convert relevant columns to numeric
cols_to_numeric = [
    "now_cost",
    "expected_goals",
    "expected_goals_per_90",
    "minutes",
    "expected_assists",
    "expected_assists_per_90",
    "saves",
    "bonus"
]
for col in cols_to_numeric:
    players_full[col] = pd.to_numeric(players_full[col])
players_full["now_cost"] = players_full["now_cost"]/10.
players_full["bonus_per_90"] = players_full["bonus"]/players_full["minutes"]*90
players_full["yellow_cards_per_90"] = players_full["yellow_cards"]/players_full["minutes"]*90

players_full = players_full[players_full["minutes"]>250]
players_full = players_full[players_full["xMin"]>55]

from scipy.stats import poisson
players_full["def_contrib_per90"] = players_full["defensive_contribution"]/players_full["minutes"]*90
players_full["lambda_def"] = players_full["def_contrib_per90"] * (1 - 0.5 * players_full["xClean_sheets"])
players_full["p_10"] = 1 - poisson.cdf(9, players_full["lambda_def"])
players_full["p_12"] = 1 - poisson.cdf(11, players_full["lambda_def"])
# --- Filter by position ---
keepers   = players_full[players_full["position"] == "Goalkeeper"].copy()
defenders = players_full[players_full["position"] == "Defender"].copy()
midfielders = players_full[players_full["position"] == "Midfielder"].copy()
forwards  = players_full[players_full["position"] == "Forward"].copy()

In [4]:
def compute_expected_points(df, goal_weight, assist_weight, cs_weight=0, save_weight=0, def_weight=0, defcon1=0, defcon2=0):
    offensive_points = (
        df["expected_goals_per_90"] * goal_weight +
        df["bonus_per_90"] +
        df["expected_assists_per_90"] * assist_weight
    ) * (df["xMin"] / 90)
    defensive_points = (
        df["xClean_sheets"] * cs_weight +
        df["saves_per_90"] * save_weight -
        df["x2"] * def_weight -
        2*df["x4"] * def_weight +
        2*df["p_10"] * defcon1 +
        2*df["p_12"] * defcon2
    )
    df["expected_points"] = (
        2 +
        offensive_points +
        defensive_points -
        df["yellow_cards_per_90"]
    )
    return df

In [5]:
forwards = compute_expected_points(forwards, 4, 3, defcon2=1)
forwards[["first_name", "second_name", "team_name", "now_cost", "total_points", "expected_points",
          "goals_scored", "assists", "expected_goals_per_90", "expected_assists_per_90", "bonus_per_90"]].sort_values(by="expected_points", ascending=False).head(15)

,first_name,second_name,team_name,now_cost,total_points,expected_points,goals_scored,assists,expected_goals_per_90,expected_assists_per_90,bonus_per_90
475,Erling,Haaland,Man City,14.8,85,8.436837,11,1,1.03,0.07,2.328590
303,Jean-Philippe,Mateta,Crystal Palace,7.7,39,5.791023,5,0,0.92,0.06,0.346154
554,Nick,Woltemade,Newcastle,7.5,34,5.334896,4,0,0.57,0.04,1.129707
193,Igor Thiago,Nascimento Rodrigues,Brentford,6.2,45,4.166211,6,0,0.57,0.02,0.728745
441,Hugo,Ekitiké,Liverpool,8.6,36,4.150200,3,1,0.42,0.04,1.262525
701,Jarrod,Bowen,West Ham,7.8,42,3.863727,3,1,0.11,0.13,1.000000
517,Benjamin,Sesko,Man Utd,7.3,28,3.857314,2,1,0.33,0.03,0.910931
31,Viktor,Gyökeres,Arsenal,9.0,30,3.829009,3,0,0.43,0.06,0.238095
267,João Pedro,Junqueira de Jesus,Chelsea,7.5,43,3.614070,2,3,0.19,0.05,0.951123
405,Dominic,Calvert-Lewin,Leeds,5.5,22,3.540249,1,1,0.30,0.08,0.337079


In [6]:
midfielders = compute_expected_points(midfielders, 5, 3, 1, defcon2=1)
midfielders[["first_name", "second_name", "team_name", "now_cost", "total_points", "expected_points", "goals_scored", "assists",
             "expected_goals_per_90", "expected_assists_per_90", "bonus_per_90", "xClean_sheets"]].sort_values(by="expected_points", ascending=False).head(15)

,first_name,second_name,team_name,now_cost,total_points,expected_points,goals_scored,assists,expected_goals_per_90,expected_assists_per_90,bonus_per_90,xClean_sheets
499,Bruno,Borges Fernandes,Man Utd,8.9,40,6.325471,2,1,0.48,0.18,0.900000,0.27
287,Ismaïla,Sarr,Crystal Palace,6.5,40,5.406464,3,1,0.48,0.04,0.882353,0.32
256,Enzo,Fernández,Chelsea,6.7,43,5.226311,3,2,0.49,0.12,0.381356,0.24
134,Antoine,Semenyo,Bournemouth,8.1,73,5.218827,6,4,0.42,0.05,1.000000,0.11
480,Bryan,Mbeumo,Man Utd,8.2,51,5.209761,4,1,0.34,0.24,0.929032,0.27
14,Bukayo,Saka,Arsenal,10.0,30,5.050162,2,0,0.24,0.36,0.359281,0.53
159,Dango,Ouattara,Brentford,6.0,33,4.870465,2,1,0.48,0.07,0.756303,0.18
427,Cody,Gakpo,Liverpool,7.6,42,4.648332,3,2,0.36,0.24,0.255319,0.33
257,Estêvão,Almeida de Oliveira Gonçalves,Chelsea,6.5,24,4.629562,1,2,0.51,0.39,0.588235,0.24
19,Declan,Rice,Arsenal,6.7,50,4.545656,1,4,0.11,0.23,0.883590,0.53


In [7]:
defenders = compute_expected_points(defenders, 6, 3, 4, def_weight=1, defcon1=1)
defenders[["first_name", "second_name", "team_name", "now_cost", "total_points",
           "expected_points", "goals_scored", "assists", "expected_goals_per_90",
           "expected_assists_per_90", "bonus_per_90", "xClean_sheets"]].sort_values(by="expected_points", ascending=False).head(15)

,first_name,second_name,team_name,now_cost,total_points,expected_points,goals_scored,assists,expected_goals_per_90,expected_assists_per_90,bonus_per_90,xClean_sheets
4,Gabriel,dos Santos Magalhães,Arsenal,6.5,68,6.115803,1,1,0.09,0.02,1.222222,0.53
7,Jurriën,Timber,Arsenal,6.1,60,6.110212,2,2,0.24,0.08,0.898716,0.53
6,Riccardo,Calafiori,Arsenal,5.8,54,5.157384,1,2,0.25,0.02,0.388489,0.53
633,Nordi,Mukiele,Sunderland,4.1,46,4.941058,1,0,0.05,0.04,0.714286,0.30
416,Virgil,van Dijk,Liverpool,6.0,35,4.835700,0,0,0.05,0.02,0.444444,0.33
552,Malick,Thiaw,Newcastle,4.9,22,4.682118,0,0,0.15,0.06,0.398230,0.37
281,Marc,Guéhi,Crystal Palace,4.9,54,4.543492,1,3,0.10,0.11,0.333333,0.32
5,William,Saliba,Arsenal,6.0,32,4.462503,0,0,0.02,0.03,0.165441,0.53
632,Omar,Alderete,Sunderland,4.1,45,4.454569,1,1,0.08,0.05,0.539730,0.30
526,Sven,Botman,Newcastle,4.9,29,4.365537,0,0,0.02,0.04,0.737705,0.37


In [8]:
keepers = compute_expected_points(keepers, 0, 0, 4, (1/3.), 1)
keepers[["first_name", "second_name", "team_name", "now_cost", "total_points",
           "expected_points", "bonus_per_90", "xClean_sheets", "saves_per_90"]].sort_values(by="expected_points", ascending=False).head(15)

,first_name,second_name,team_name,now_cost,total_points,expected_points,bonus_per_90,xClean_sheets,saves_per_90
0,David,Raya Martín,Arsenal,5.7,46,4.812760,0.333333,0.53,1.78
630,Robin,Roefs,Sunderland,4.7,52,4.635596,0.666667,0.30,3.67
518,Senne,Lammens,Man Utd,5.0,14,4.571710,0.666667,0.27,3.67
521,Nick,Pope,Newcastle,5.2,46,4.395902,0.333333,0.37,2.89
346,Bernd,Leno,Fulham,5.0,25,4.393842,0.111111,0.40,3.11
410,Giorgi,Mamardashvili,Liverpool,4.3,5,4.212806,0.000000,0.33,3.67
312,Jordan,Pickford,Everton,5.5,33,4.079521,0.333333,0.31,2.56
637,Guglielmo,Vicario,Spurs,5.1,42,3.909260,0.555556,0.22,2.89
196,Bart,Verbruggen,Brighton,4.4,14,3.819256,0.000000,0.34,2.33
476,Gianluigi,Donnarumma,Man City,5.7,23,3.583951,0.000000,0.38,1.50


In [9]:
players_full.loc[players_full["position"] == "Forward", "expected_points"] = forwards["expected_points"].values
players_full.loc[players_full["position"] == "Midfielder", "expected_points"] = midfielders["expected_points"].values
players_full.loc[players_full["position"] == "Defender", "expected_points"] = defenders["expected_points"].values
players_full.loc[players_full["position"] == "Goalkeeper", "expected_points"] = keepers["expected_points"].values

In [10]:
players_full[["first_name", "second_name", "team_name", "now_cost", "xMin", "total_points",
           "expected_points"]].sort_values(by="expected_points", ascending=False).head(30)

,first_name,second_name,team_name,now_cost,xMin,total_points,expected_points
475,Erling,Haaland,Man City,14.8,87.0,85,8.436837
499,Bruno,Borges Fernandes,Man Utd,8.9,89.0,40,6.325471
4,Gabriel,dos Santos Magalhães,Arsenal,6.5,90.0,68,6.115803
7,Jurriën,Timber,Arsenal,6.1,81.0,60,6.110212
303,Jean-Philippe,Mateta,Crystal Palace,7.7,86.0,39,5.791023
287,Ismaïla,Sarr,Crystal Palace,6.5,85.0,40,5.406464
554,Nick,Woltemade,Newcastle,7.5,85.0,34,5.334896
256,Enzo,Fernández,Chelsea,6.7,87.0,43,5.226311
134,Antoine,Semenyo,Bournemouth,8.1,88.0,73,5.218827
480,Bryan,Mbeumo,Man Utd,8.2,79.0,51,5.209761


In [13]:
import requests
import pandas as pd

player_id = 633  # Erling Haaland 430
url = f"https://fantasy.premierleague.com/api/element-summary/{player_id}/"
data = requests.get(url).json()

# --- Past season summary ---
past = pd.DataFrame(data["history_past"])
print(past[["season_name", "total_points", "minutes", "goals_scored", "assists", "clean_sheets"]])

  season_name  total_points  minutes  goals_scored  assists  clean_sheets
0     2020/21             0        0             0        0             0
1     2021/22             0        0             0        0             0
2     2022/23            41     1307             0        1             3
3     2023/24            29      727             0        0             2
4     2024/25             0        0             0        0             0


In [14]:
past[past["minutes"]!=0]

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,...,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
2,2022/23,490721,40,38,41,1307,0,1,3,29,...,57.2,0,0,0,0,16,0.17,1.00,1.17,22.62
3,2023/24,490721,45,44,29,727,0,0,2,16,...,29.0,0,0,0,0,7,0.31,0.68,0.99,12.99


In [32]:
def history(players_full, id):
  url = f"https://fantasy.premierleague.com/api/element-summary/{id}/"
  data = requests.get(url).json()
  past = pd.DataFrame(data["history_past"])
  if past.empty: # Add this check
      return 0
  past = past[past["season_name"].isin(["2022/23","2023/24","2024/25"])]
  past = past[past["minutes"]>250]
  if(past.shape[0]!=0):
    past["g"] = past["goals_scored"]/past["minutes"]
    past["a"] = past["assists"]/past["minutes"]
    past["b"] = past["bonus"]/past["minutes"]
    past["s"] = past["saves"]/past["minutes"]
    past["y"] = past["yellow_cards"]/past["minutes"]
    past["d"] = past["defensive_contribution"]/past["minutes"]
    if(past.shape[0]==1):
      return [past["g"].iloc[0],past["a"].iloc[0],past["b"].iloc[0],past["s"].iloc[0],past["y"].iloc[0],past["d"].iloc[0],id]
    elif(past.shape[0]==2):
      return [past["g"].iloc[0]*.35+past["g"].iloc[1]*.65,past["a"].iloc[0]*.35+past["a"].iloc[1]*.65,
                  past["b"].iloc[0]*.35+past["b"].iloc[1]*.65,past["s"].iloc[0]*.35+past["s"].iloc[1]*.65,
                  past["y"].iloc[0]*.35+past["y"].iloc[1]*.65,past["d"].iloc[0]*.35+past["d"].iloc[1]*.65,id]
    else:
      return [past["g"].iloc[0]*.1+past["g"].iloc[1]*.25+past["g"].iloc[2]*.65, past["a"].iloc[0]*.1+past["a"].iloc[1]*.25+past["a"].iloc[2]*.65,
                  past["b"].iloc[0]*.1+past["b"].iloc[1]*.25+past["b"].iloc[2]*.65, past["s"].iloc[0]*.1+past["s"].iloc[1]*.25+past["s"].iloc[2]*.65,
                  past["y"].iloc[0]*.1+past["y"].iloc[1]*.25+past["y"].iloc[2]*.65, past["d"].iloc[0]*.1+past["d"].iloc[1]*.25+past["d"].iloc[2]*.65,id]
  else:
    return 0

In [33]:
g_list = []
for id in players_full['player_id']:
  yut = history(players_full, id)
  if(yut!=0):
    g_list.append(yut)

In [34]:
g_list

[[np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0030062134502923978),
  np.float64(0.024841008771929823),
  np.float64(0.0007730263157894738),
  np.float64(0.0),
  1],
 [np.float64(0.0011597728454016622),
  np.float64(0.0006323308912911439),
  np.float64(0.003573140455173012),
  np.float64(0.0),
  np.float64(0.0015756979094452984),
  np.float64(0.04373677528565383),
  5],
 [np.float64(0.0006567868589166092),
  np.float64(0.00011450728269950239),
  np.float64(0.004524408597211586),
  np.float64(0.0),
  np.float64(0.000885801424315614),
  np.float64(0.04619940769990128),
  6],
 [np.float64(0.002044989775051125),
  np.float64(0.0010224948875255625),
  np.float64(0.0010224948875255625),
  np.float64(0.0),
  np.float64(0.00408997955010225),
  np.float64(0.06748466257668712),
  7],
 [np.float64(0.00041407867494824016),
  np.float64(0.0012422360248447205),
  np.float64(0.0037267080745341614),
  np.float64(0.0),
  np.float64(0.002898550724637681),
  np.float64(0.051759834368530024),
  8],

In [35]:
g_data = pd.DataFrame(g_list, columns=["g","a","b","s","y","d","id"])

In [43]:
g_data.sort_values(by="g",ascending=False).head(10)

,g,a,b,s,y,d,id
107,0.009172,0.001821,0.010462,0.0,0.000754,0.017343,430
97,0.007942,0.005048,0.013665,0.0,0.000451,0.026200,381
17,0.005964,0.003658,0.011071,0.0,0.000940,0.019553,64
69,0.005471,0.001262,0.009462,0.0,0.001223,0.028047,283
155,0.005412,0.001546,0.006571,0.0,0.001546,0.035176,654
84,0.005399,0.002700,0.005662,0.0,0.002872,0.083798,328
108,0.005266,0.002917,0.007707,0.0,0.000999,0.051962,119
114,0.005194,0.002470,0.012428,0.0,0.002028,0.050077,450
98,0.005089,0.002663,0.009556,0.0,0.001845,0.046018,384
59,0.004887,0.002520,0.010746,0.0,0.001680,0.032066,249


In [50]:
players_full[players_full["player_id"]==119][["first_name","second_name"]]

,first_name,second_name
480,Bryan,Mbeumo
